# 코드설명(임시)

- source embedding 추출하는 코드
- source pca

In [1]:
import tqdm

def nop(it, *a, **k):
    return it

real_tqdm = tqdm.tqdm
tqdm.tqdm = nop
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import pickle
import cv2
import time
import glob

import numpy as np
np.bool = np.bool_
import torch
import torch.nn.functional as F
from sklearn.decomposition import PCA

from utils.inference.faceshifter_run import faceshifter_batch
from utils.inference.image_processing import crop_face, get_final_image, show_images, normalize_and_torch, normalize_and_torch_batch
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement, crop_frames_and_get_transforms, resize_frames
from utils.inference.core import model_inference, transform_target_to_torch

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions




/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:143: DeprecationWarning: In accordance with NEP 32, the function mirr was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  mirr = onp.mirr
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:160: DeprecationWarning: In accordance with NEP 32, the function npv was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  npv = onp.npv
/compuworks/anaconda3/envs/xaise

### Load Models

In [2]:
app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
loading ./coordinate_reg/model/2d106det 0
input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)


[05:27:14] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.5.0. Attempting to upgrade...
[05:27:14] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[05:27:14] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8302 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


Network [LIPSPADEGenerator] was created. Total number of parameters: 72.2 million. To see the architecture, do print(network).
Load checkpoint from path:  weights/10_net_G.pth


# Source embeding Extraction

In [3]:
from sklearn.decomposition import PCA
import pickle

In [4]:
crop_size = 224
np.random.seed(7777)

dataset_dir = "./examples/images/VggFace2-crop"
save_dir = "./examples/arc_embeds/VggFace2-crop"
os.makedirs(save_dir, exist_ok=True)

id_folder_list = glob.glob(f"{dataset_dir}/*")
id_folder_list.sort()
with open(f"{save_dir}/arc_embed_list.txt", "w") as embed_log:
    output_cnt = 0
    for id_folder in real_tqdm(id_folder_list):
        pic_names = glob.glob(id_folder+"/*.jpg")
        pic_names.sort()
        for pic_name in pic_names:
            source_full = cv2.imread(pic_name)
            # source image는 rgb여야 함.
            source_curr = cv2.resize(source_full[:, :, ::-1], (crop_size, crop_size))

            source_curr = normalize_and_torch(source_curr)
            source_embed = netArc(F.interpolate(source_curr, scale_factor=0.5, mode='bilinear', align_corners=True)).cpu().detach().numpy()
            
            np.save(f"{save_dir}/{str(output_cnt)}.npy", source_embed[0])
            embed_log.write(pic_name+"\n")
            output_cnt += 1


  6%|▌         | 485/8631 [58:56<13:02:42,  5.77s/it]

In [ ]:
pca_dir = "./pca_pkl"

embeds_path_list = glob.glob(f"{save_dir}/*.npy")
embeds_path_list.sort()

exp_emb = np.load(embeds_path_list[0])

embed_array = np.zeros([len(embeds_path_list), exp_emb.shape[0]], dtype=exp_emb.dtype)

for i, embed_path in enumerate(embeds_path_list):
    embed_array[i] = np.load(embed_path)

pca = PCA(svd_solver="full")

pca.fit(embed_array)

with open(f"{pca_dir}/netArcPCA.pkl", "wb") as file:
    pickle.dump(pca, file)

pca_array = pca.transform(embed_array)

pca_minmax = dict({"min": np.amin(pca_array, axis=0), "max": np.amax(pca_array, axis=0)})

with open(f"{pca_dir}/netArcPCAMinMax.pkl", "wb") as file:
    pickle.dump(pca_minmax, file)


PCA(svd_solver='full')